<a href="https://colab.research.google.com/github/braxtonb46/fraud-clustering/blob/main/Fraud_Clusters_%26_Classification_HFCLASS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
!pip install kaggle
!mkdir -p ~/.kaggle

#Create kaggle.json file (you'll need to replace these with your actual Kaggle credentials)
!echo '{XXX}' > ~/.kaggle/kaggle.json

# Set proper permissions
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle datasets download -d mlg-ulb/creditcardfraud

!unzip creditcardfraud.zip

import pandas as pd
df = pd.read_csv('creditcard.csv')


Dataset URL: https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud
License(s): DbCL-1.0
 85% 56.0M/66.0M [00:00<00:00, 141MB/s]
100% 66.0M/66.0M [00:00<00:00, 126MB/s]
Archive:  creditcardfraud.zip
  inflating: creditcard.csv          


In [3]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import seaborn as sns


In [4]:
!pip install transformers

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [89]:
import pandas as pd
import torch
from torch import nn
from transformers import Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.metrics import precision_score, recall_score, f1_score

In [77]:
import torch.nn.functional as F

In [6]:
# Example text-based model (adapt for text features)
model_name = 'bert-base-uncased'  # Change as necessary
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
#print(df.info)
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
fraud = df[df['Class'] == 1]
non_fraud = df[df['Class'] == 0].sample(len(fraud), random_state=42)
balanced_df = pd.concat([fraud, non_fraud])

In [8]:
x = balanced_df.drop(columns = 'Class')
y = balanced_df['Class']


#Classification - Torch NN

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42, stratify=y
    )

In [57]:
from torch.utils.data import Dataset

class FraudDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features.values, dtype=torch.float32)
        self.labels = torch.tensor(labels.values, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {'input_ids': self.features[idx], 'labels': self.labels[idx]}


In [58]:
train_dataset = FraudDataset(X_train, y_train)
test_dataset = FraudDataset(X_test, y_test)

In [79]:
class TabularClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, output_dim=2):
        super(TabularClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim//2)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_dim//2, output_dim)

    def forward(self, input_ids, labels = None):
        x = self.fc1(input_ids)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        logits = self.fc3(x)


        # Calculate loss if labels are provided
        loss = None
        if labels is not None:
            loss = F.cross_entropy(logits, labels)  # Use cross-entropy loss

        return loss, logits
        #return logits


In [80]:
input_dim = X_train.shape[1]  # Number of features
model = TabularClassifier(input_dim=input_dim)

In [90]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    #precision = precision_score(labels,preds)
    #recall = recall_score(labels,preds)
    #f1 = f1_score(labels,preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }


In [91]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=15,               #adjust the number of epochs
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,                  # Strength of weight decay
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [92]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)


In [94]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.000000,0.275246,0.998280,0.000000,0.000000,0.000000
2,0.000000,0.184874,0.998280,0.000000,0.000000,0.000000
3,0.863700,0.162347,0.998280,0.000000,0.000000,0.000000
4,0.067600,0.055651,0.998280,0.000000,0.000000,0.000000
5,0.000000,0.142831,0.998280,0.000000,0.000000,0.000000
6,0.000000,0.134399,0.998297,0.526316,0.102041,0.170940
7,0.000000,0.118564,0.998315,1.000000,0.020408,0.040000
8,0.000000,0.079361,0.998350,0.642857,0.091837,0.160714
9,0.000100,0.075744,0.998402,0.640000,0.163265,0.260163
10,0.072400,0.041787,0.998508,0.675676,0.255102,0.370370


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.p

TrainOutput(global_step=106815, training_loss=0.11810546212193918, metrics={'train_runtime': 614.9772, 'train_samples_per_second': 5557.402, 'train_steps_per_second': 173.689, 'total_flos': 0.0, 'train_loss': 0.11810546212193918, 'epoch': 15.0})

In [85]:
results = trainer.evaluate()
print(results)


{'eval_loss': 1.0857688188552856, 'eval_accuracy': 0.9982795547909132, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_runtime': 1.7359, 'eval_samples_per_second': 32813.865, 'eval_steps_per_second': 513.275, 'epoch': 15.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [86]:

predictions = trainer.predict(test_dataset)
preds = predictions.predictions.argmax(-1)


results_df = X_test.copy()
results_df['Actual'] = y_test.values
results_df['Predicted'] = preds


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [97]:
results_df[['Actual' , 'Predicted']].head(5)

,Actual,Predicted
263020,0,0
11378,0,0
147283,0,0
219439,0,0
36939,0,0
